# Baseline Methods

## Imports 

In [ ]:
use_google_drive = False

try:
  import google.colab
  from google.colab import drive
  !pip install webdataset
  use_google_drive = True
except Exception:
  pass


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import webdataset as wds
import cv2
import os
import ipyparallel as ipp
from IPython.display import display, HTML
from jinja2 import Environment

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import math

from numpy import unique
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin, pairwise_distances
from colorthief import ColorThief
from PIL import Image
import io
import imagequant
import time
import colorsys

from logging import exception


## Definitions and Parameters

In [ ]:
dataset_id = 3
no_multicolor = True
color_space = 'hsl' #rgb, hsv or hsl


In [ ]:
color_tag = 'singlecolor' if no_multicolor else 'multicolor'

if use_google_drive:
  dataset_file = f"file:///content/gdrive/MyDrive/ColabData/amazon/shoes-224-full-{dataset_id}.tar"
  pairwise_file = f"file:///content/gdrive/MyDrive/ColabData/amazon/shoes-224-{dataset_id}-{color_tag}-{color_space}.npz"
  dataset_folder = "file:///content/gdrive/MyDrive/ColabData/amazon/"
  report_folder = f"file:///content/gdrive/MyDrive/ColabData/amazon/report/dataset-{dataset_id}/{color_tag}-{color_space}"

  drive.mount("/content/gdrive")
else:
  dataset_file = f"file://{os.getcwd()}/dataset/shoes-224-full-{dataset_id}.tar".replace('\\', '/')
  pairwise_file = f"./dataset/shoes-224-{dataset_id}-{color_tag}-{color_space}.npz"
  dataset_folder = "./dataset"
  report_folder = f"./report/dataset-{dataset_id}/{color_tag}-{color_space}"


In [ ]:
classes = ["black", "white", "gray", "red", "green", "blue", "orange", "purple", "yellow", "pink", "brown", "multicolor"]

classpoints = np.array([
    [0, 0, 0], #black
    [255, 255, 255], #white
    [128, 128, 128], #gray
    [255, 0, 0], #red
    [0, 255, 0], #green
    [0, 0, 255], #blue
    [255, 165, 0], #orange
    [128, 0, 128], #purple
    [255, 255, 0], #yellow
    [255, 192, 203], #pink
    [165, 42, 42], #brown
])


if (color_space == 'rgb'):
  max_distance = 255 * math.sqrt(3)
elif (color_space == 'hsv'):
  max_distance = 2.0
elif (color_space == 'hsl'):
  max_distance = 2.0
else:
  raise exception("unsupported colorspace")


## Helper Functions

In [ ]:
def load_cache(key):
  file = f"{dataset_folder}/cache-{key}.npz"
  if os.path.exists(file):
    cache = np.load(file, allow_pickle=True)
    cache = cache['cache'].item()
    print(f"{key} cache loaded from {file}, {len(cache)} keys total")
    return cache
  else:
    return dict()

def save_cache(input: dict, key):
  file = f"{dataset_folder}/cache-{key}.npz"
  np.savez_compressed(file, cache=input)
  print(f"{key} cache saved to {file}, {len(input)} keys total")

def combine_save_cache(inputs: list, key):
  ret = set()
  for input in inputs:
    ret = ret.union(input.items())
  ret = dict(ret)
  save_cache(ret, key)
  return ret


In [ ]:
def find_base_color(prob_dist):
  if not no_multicolor:
    base_colors = np.array(np.where(prob_dist >= 0.25)).reshape(-1)
    if len(base_colors) == 1:
      return base_colors[0]
    else:
      return 11 #multicolor
  else:
    return np.argmax(prob_dist)

def convert_distance_probabilities(dists):
  ratios = np.array(dists / max_distance)
  ratios = np.where(ratios > 0.0, ratios, ratios + (1 / 709.78)) #709.78 is the exp's max value
  ratios = 1 / ratios
  return np.around(np.exp(ratios) / np.sum(np.exp(ratios), axis=0), 2)

def convert_classarray_probabilities(class_array):
  classesfound, countsfound = np.unique(class_array, return_counts=True)
  counts = []
  for c in range(len(classpoints)):
    if c in classesfound:
      counts.append(float(countsfound[np.where(classesfound == c)][0]))
    else:
      counts.append(0.0)
  return np.around(np.array(counts) / np.sum(countsfound), 2)


In [ ]:
#color conversions
@ipp.require('colorsys')
def rgb_to_hsl(r, g, b):
  h, l, s = colorsys.rgb_to_hls(r / 255, g / 255, b / 255)
  if s < 0.05 or l < 0.05:
    h = 0
  return round(h * 360), round(s, 2), round(l, 2)

@ipp.require('colorsys', round)
def rgb_to_hsv(r, g, b):
  h, s, v = colorsys.rgb_to_hsv(r / 255, g / 255, b / 255)
  if s < 0.05 or v < 0.05:
    h = 0
  return round(h * 360), round(s, 2), round(v, 2)


def hsv_to_hsl(h, s, v):
  l = v * (1 - (s / 2))
  s2 = 0
  if l != 1 and l != 0:
    s2 = (v - l) / min(l, 1 - l)
  return h, round(s2, 2), round(l, 2)

def hsl_to_hsv(h, s, l):
  v = l + s * min(l, 1 - l)
  s2 = 0
  if v != 0:
    s2 = 2 * (1 - (l / v))
  return h, round(s2, 2), round(v, 2)


hsv_cache = load_cache("rgb-hsv")
hsl_cache = load_cache("rgb-hsl")

@ipp.require('math')
def hcz_to_xyz(h, c, z):
  return math.cos(h * math.pi / 180) * c, math.sin(h * math.pi / 180) * c, z

@ipp.require(rgb_to_hsv)
def rgb_to_hsv_cache(rgb):
  r, g, b = rgb
  if (r, g, b) not in hsv_cache:
    h, s, v = rgb_to_hsv(r, g, b)
    hsv_cache[(r, g, b)] = hcz_to_xyz(h, s * v, v)

  return hsv_cache[(r, g, b)]

@ipp.require(rgb_to_hsl, hcz_to_xyz)
def rgb_to_hsl_cache(rgb):
  r, g, b = rgb
  if (r, g, b) not in hsl_cache:
    h, _, l = rgb_to_hsl(r, g, b)
    _, s, v = rgb_to_hsv(r, g, b)
    hsl_cache[(r, g, b)] = hcz_to_xyz(h, s * v, l)
  return hsl_cache[(r, g, b)]


In [ ]:
#convert rgb classpoints to target colorspace

if (color_space == 'hsv'):
  classpoints = [rgb_to_hsv_cache([r, g, b]) for r, g, b in classpoints]
elif (color_space == 'hsl'):
  classpoints = [rgb_to_hsl_cache([r, g, b]) for r, g, b in classpoints]

print(f"Classpoints converted to {color_space} coordinates")
print(classpoints)


In [ ]:
#distance calculation
cone_euclidean_cache = load_cache("cone-euclidean")

@ipp.require('math')
def cone_euclidean(h1, h2, s1, s2, v1, v2, z1, z2):
  if (h1, h2, s1, s2, v1, v2, z1, z2) not in cone_euclidean_cache:
    p1 = (z2 - z1)**2
    p2 = (s1**2 * v1**2)
    p3 = (s2**2 * v2**2)
    cos = math.cos((h2 - h1) * math.pi / 180)
    p4 = 2 * s1 * v1 * s2 * v2 * cos

    p = round(p1 + p2 + p3 - p4, 6)
    cone_euclidean_cache[(h1, h2, s1, s2, v1, v2, z1, z2)] = math.sqrt(p)

  return cone_euclidean_cache[(h1, h2, s1, s2, v1, v2, z1, z2)]

@ipp.require(cone_euclidean)
def cone_distance(x, y):
  h1, s1, v1 = x[0], x[1], x[2]
  h2, s2, v2 = y[0], y[1], y[2]

  if (h1 == h2) and (s1 == s2) and (v1 == v2):
    return 0

  return cone_euclidean(h1, h2, s1, s2, v1, v2, v1, v2)

@ipp.require(cone_euclidean)
def bicone_distance(x, y):
  h1, s1, l1, sa1, v1 = x[0], x[1], x[2], x[3], x[4]
  h2, s2, l2, sa2, v2 = y[0], y[1], y[2], y[3], y[4]

  if (h1 == h2) and (s1 == s2) and (l1 == l2):
    return 0

  return cone_euclidean(h1, h2, sa1, sa2, v1, v2, l1, l2)

def euclidean(x, y):
  r1, g1, b1 = x[0], x[1], x[2]
  r2, g2, b2 = y[0], y[1], y[2]

  if (r1 == r2) and (g1 == g2) and (b1 == b2):
    return 0

  return np.float16(math.sqrt((r2 - r1)**2 + (g2 - g1)**2 + (b2 - b1)**2))


@ipp.require(bicone_distance, pairwise_distances, euclidean)
def pairwise_distances_colorspace(colors_array):
  return np.float16(pairwise_distances(colors_array, classpoints, metric='euclidean'))


## Load Dataset

In [ ]:
dataset = (wds.WebDataset(dataset_file)
           .select(predicate=lambda r: not no_multicolor or int(r["cls"]) != 11)
           .decode("pil", only="jpg")
           .to_tuple("jpg", "cls")
           )

i = 0
for jpg, cls in dataset:
  i += 1
  if i == 1:
    break

print(classes[int(cls)])
plt.imshow(jpg)
plt.show()


## Image Helper Functions

In [ ]:
@ipp.require('numpy as np', 'cv2')
def remove_bg(jpg, show_result=False):
  image = cv2.cvtColor(np.array(jpg), cv2.COLOR_RGB2RGBA)

  gray = (cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)).astype(np.uint8)

  ret, thresh = cv2.threshold(gray,
                              int(image[:, :, 0].mean()),
                              int(image[:, :, 1].mean()),
                              0)

  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

  mask = np.zeros(shape=(gray.shape), dtype=np.uint8)
  cv2.drawContours(mask, contours, -1, (1, 0, 0), cv2.FILLED)
  mask = (~(mask == 1) * 1).astype(np.uint8)

  img_masked = cv2.bitwise_and(image, image, mask=mask)
  if show_result:
    plt.figure(figsize=(20, 60))
    plt.subplot(131)
    plt.imshow(image)
    plt.subplot(132)
    plt.imshow(mask)
    plt.subplot(133)
    plt.imshow(img_masked)

  return img_masked


img_masked = remove_bg(jpg, show_result=True)


In [ ]:
@ipp.require('numpy as np', rgb_to_hsv_cache, rgb_to_hsl_cache)
def flatten_colors(img_array):
  result = np.array(img_array).reshape(-1, 4)
  result = result[result[:, 3] == 255]
  result = np.delete(result, 3, 1)

  if color_space == 'rgb':
    return result
  elif color_space == 'hsv':
    return np.apply_along_axis(rgb_to_hsv_cache, 1, result)
  elif color_space == 'hsl':
    return np.apply_along_axis(rgb_to_hsl_cache, 1, result)
  else:
    raise exception('unsupported colorspace')


print("Flatten pixels of the image")
ca = flatten_colors(img_masked)
print(ca)
pwdc = pairwise_distances_colorspace(ca)


## Quantization Methods

In [ ]:
def quantization_MMCQ(img_array):
  with io.BytesIO() as file_object:
    img = Image.fromarray(np.uint8(img_array)).convert('RGBA')
    img.save(file_object, "PNG")
    cf = ColorThief(file_object)
    color = cf.get_color()
    dists = pairwise_distances([color], classpoints).reshape(-1)
    return convert_distance_probabilities(dists)

def quantization_FASTOCTREE(img_array):
  img = Image.fromarray(np.uint8(img_array)).convert('RGBA')
  img = img.quantize(2, method=Image.FASTOCTREE)
  palette = np.array(img.getpalette())
  color = np.delete(palette, np.where(palette == 0))
  if (len(color) == 0):
    color = [1, 1, 1]
  dists = pairwise_distances([[color[0], color[1], color[2]]], classpoints).reshape(-1)
  return convert_distance_probabilities(dists)

def quantization_libimagequant(img_array):
  img = Image.fromarray(np.uint8(img_array)).convert('RGBA')
  img = imagequant.quantize_pil_image(img, dithering_level=1.0, max_colors=1)
  palette = np.array(img.getpalette())
  color = np.delete(palette, np.where(palette == 0))
  dists = pairwise_distances([color], classpoints).reshape(-1) + 1
  return convert_distance_probabilities(dists)


In [ ]:
if color_space == 'rgb':
  print(quantization_MMCQ(img_masked))
  print(quantization_FASTOCTREE(img_masked))
  print(quantization_libimagequant(img_masked))
else:
  print(f"quantization methods are not available in {color_space} color space")


## Pairwise Distance Quantile Methods

In [ ]:
def pairwise_quantile(distances, quantile=0.5):
  dists = np.quantile(distances, quantile, axis=0)
  return convert_distance_probabilities(dists)

def pairwise_first_quantile(distances):
  return pairwise_quantile(distances, 0.25)

def pairwise_median(distances):
  return pairwise_quantile(distances, 0.5)

def pairwise_third_quantile(distances):
  return pairwise_quantile(distances, 0.75)


In [ ]:
print(pairwise_first_quantile(pwdc))
print(pairwise_median(pwdc))
print(pairwise_third_quantile(pwdc))


## Pairwise Distance Methods

In [ ]:
def pairwise_argmin(distances):
  pwa = np.argmin(distances, axis=1).reshape(-1)
  return convert_classarray_probabilities(pwa)

def pairwise_mean(distances):
  dists = np.average(distances, axis=0)
  return convert_distance_probabilities(dists)

def array_mean_argmin(colors_array):
  center = np.round(np.mean(colors_array, axis=0), 0).reshape(1, -1)
  pwd = pairwise_distances_colorspace(center)
  return pairwise_argmin(pwd)


In [ ]:
print(pairwise_argmin(pwdc))
print(pairwise_mean(pwdc))
print(array_mean_argmin(ca))


## Processing Functions

In [ ]:
def process_colors_array(colors_array, decision_method=array_mean_argmin):
  prob_dist = decision_method(colors_array)
  return find_base_color(prob_dist)

def process_distances(distances, decision_method=pairwise_argmin):
  prob_dist = decision_method(distances)
  return find_base_color(prob_dist)

def process_image(img_array, decision_method=quantization_MMCQ):
  prob_dist = decision_method(img_array)
  return find_base_color(prob_dist)


In [ ]:
if color_space == 'rgb':
  print("\nQuantization method results for sample image")
  print("quantization_MMCQ: ", classes[process_image(img_masked, decision_method=quantization_MMCQ)])
  print("quantization_FASTOCTREE: ", classes[process_image(img_masked, decision_method=quantization_FASTOCTREE)])
  print("quantization_libimagequant: ", classes[process_image(img_masked, decision_method=quantization_libimagequant)])
else:
  print(f"Quantization methods only supports RGB colorspace")

print("\nQuantile method results for sample image")
print("first quantile: ", classes[process_distances(pwdc, decision_method=pairwise_first_quantile)])
print("second quantile: ", classes[process_distances(pwdc, decision_method=pairwise_median)])
print("third quantile: ", classes[process_distances(pwdc, decision_method=pairwise_third_quantile)])

print("\nPairwise distance method results for sample image")
print("Pairwise ArgMin: ", classes[process_distances(pwdc, decision_method=pairwise_argmin)])
print("Pairwise Mean: ", classes[process_distances(pwdc, decision_method=pairwise_mean)])
print("Array Mean: ", classes[process_colors_array(ca, decision_method=array_mean_argmin)])


## Process Dataset

In [ ]:
results = []


### Process with Quantization Methods

In [ ]:
if color_space == 'rgb':
  methods = [quantization_FASTOCTREE, quantization_MMCQ, quantization_libimagequant]

  for method in methods:
    t = time.time()

    i = 0
    y = []
    yhat = []

    for jpg, cls in dataset:
      i += 1

      img_masked = remove_bg(jpg, show_result=False)
      pwd = process_image(img_masked, decision_method=method)

      y.append(int(cls))
      yhat.append(pwd)

    elapsed = time.time() - t
    results.append((method.__name__, elapsed, y, yhat))

    print(f"method: {method.__name__} \nelapsed time: {elapsed:.2f} s")
    print(f"accuracy: {accuracy_score(y,yhat):.2f}")
    print(60 * "_")
else:
  print(f"Quantization methods only supports RGB colorspace")


### Calculate and Save Pairwise Distances

In [ ]:
pwdcs = []

if os.path.exists(pairwise_file):
  npz = np.load(pairwise_file, allow_pickle=True)
  pwdcs = npz["pwdcs"]

  print(f"pairwise distances loaded from {pairwise_file}")
else:
  print(f"pairwise distances file is missing, they will be calculated and saved to {pairwise_file}")


In [ ]:
@ipp.require(remove_bg, flatten_colors, pairwise_distances_colorspace)
def calculate_pairwise_distances(input):
  jpg, cls = input
  img_masked = remove_bg(jpg, show_result=False)
  ca = flatten_colors(img_masked)
  pwdc = pairwise_distances_colorspace(ca)
  return cls, ca, pwdc


if len(pwdcs) == 0:
  with ipp.Cluster() as rc:
    dview = rc.direct_view()
    dview.push(dict(hsv_cache=hsv_cache, hsl_cache=hsl_cache, color_space=color_space,
                    classpoints=classpoints, cone_euclidean_cache=cone_euclidean_cache), block=True)

    view = rc.load_balanced_view()

    i = 0
    t = time.time()
    print("calculating pairwise distances:")

    for ret in view.imap(calculate_pairwise_distances, dataset, ordered=False, return_exceptions=True):
      if type(ret) is ipp.RemoteError:
        raise ret
      else:
        pwdcs.append(ret)

        i += 1
        if i % 100 == 0:
          print(f"{i} processed, {i/(time.time() - t):.2f} ips... ", end="")

    print(f"\nTotal {i} processed, {i/(time.time() - t):.2f} ips")

    #print(f"\nOperation completed. Now, caches are pulling, combining and saving, please wait")
    #hsv_cache = combine_save_cache(dview.pull('hsv_cache', block=True), "rgb-hsv")
    #hsl_cache = combine_save_cache(dview.pull('hsl_cache', block=True), "rgb-hsl")
    #cone_euclidean_cache = combine_save_cache(dview.pull('cone_euclidean_cache', block=True), "cone-euclidean")

    print(f"\npairwise distances are saving to {pairwise_file}")
    np.savez_compressed(pairwise_file, pwdcs=np.array(pwdcs, dtype=object))
    print(f"pairwise distances saved to {pairwise_file}")
else:
  print(f"pairwise distances is already loaded, skipping")


### Process Pairwise Distance based Methods

In [ ]:
methods = [pairwise_argmin, pairwise_mean, array_mean_argmin, pairwise_first_quantile, pairwise_median, pairwise_third_quantile]

for method in methods:
  t = time.time()

  i = 0
  y = []
  yhat = []

  for cls, ca, pwdc in pwdcs:
    i += 1

    if method.__name__ == "array_mean_argmin":
      pwd = process_colors_array(ca, decision_method=method)
    else:
      pwd = process_distances(pwdc, decision_method=method)

    y.append(int(cls))
    yhat.append(pwd)

  elapsed = time.time() - t
  results.append((method.__name__, elapsed, y, yhat))

  print(f"method: {method.__name__} \nelapsed time: {elapsed:.2f} s")
  print(f"accuracy: {accuracy_score(y,yhat):.2f}")
  print(60 * "_")


## Report Results

In [ ]:
template = """
<!DOCTYPE html>
<html lang="en">
<head>
  <style>
    html{
      font-family: "Times New Roman", Times, serif;
      font-size: 12pt;
    }
    table {
      border-spacing: 0px;
    }

    div.centered 
    {
        text-align: center;
    }

    div.centered table 
    {
        margin: 0 auto; 
        text-align: left;
    }

    th {
      text-align: center;
      border-bottom: 2px solid black;
      padding: 2px 5px 2px 5px;
    }
    td {
      text-align: right;
      padding: 5px;
    }
    img{
      text-align:center;
    }
    caption{
      padding-bottom:10px;
    }
  </style>
  <meta charset="utf-8">
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
</head>
<body>
  <div class="centered">
    <table cellspacing="0" cellpadding="0" lang="en">
      <caption>Summary<br/></caption>
      <tr>
        <th>Method</th>
        <th>Duration</th>
        <th>Accuracy</th>
      </tr>
      {% for key,value in summary.items() %}
      <tr>
        <td style="text-align: left;"><b>{{key}}</b></td>
        <td>{{"%.2f"|format(value.duration)}} sec.</td>
        <td>{{"%.4f"|format(value.accuracy)}}</td>
      </tr>
      {% endfor %}
    </table>
    {% if color_space != 'rgb' %}
    <sub>* Quantization based methods are available only in RGB color space</sub>
    {% endif %}
  </div>
{% for result in data %}
  <p><b>{{run}}</b></p>
  <p><b>{{result.title}} Result</b></p>
  <p>Total Duration: {{"%.2f"|format(result.elapsed)}} sec.</p>
  <div class="centered">
    <table cellspacing="0" cellpadding="0" lang="en">
      <caption>{{result.title}} Classification Report<br/></caption>
      <tr>
        <th></th>
        <th>Precision</th>
        <th>Recall</th>
        <th>F1 Score</th>
        <th>Support</th>
      </tr>
      {% for key,value in result.report.items() %}
      {% if key=="micro avg" or key=="accuracy" %}
      <tr>
        <td>&nbsp;</td>
        <td>&nbsp;</td>
        <td>&nbsp;</td>
        <td>&nbsp;</td>
        <td>&nbsp;</td>
      </tr>
      {% endif %}
      {% if key != "accuracy" %}
      <tr>
        <td style="text-align: left;"><b>{{key}}</b></td>
        <td>{{"%.4f"|format(value.precision) }}</td>
        <td>{{"%.4f"|format(value.recall)}}</td>
        <td>{{"%.4f"|format(value.get('f1-score'))}}</td>
        <td>{{value.support}}</td>
      </tr>
      {% else %}
      <tr>
        <td style="text-align: left;"><b>{{key}}</b></td>
        <td></td>
        <td></td>
        <td>{{"%.4f"|format(value)}}</td>
        <td></td>
      </tr>
      {% endif %}
      {% endfor %}
      </table>
    </div>
    <br clear=all style='page-break-before:always'>
    <div class="centered">
      <figure>
        <img src="{{result.method}}.png" align="center" />
        <figcaption>{{result.title}} Confusion Matrix</figcaption>
      </figure>
    </div>
    <br clear=all style='page-break-before:always'>
{% endfor %}
</body>
</html>
"""
environment = Environment()
template = environment.from_string(template)
font = {'family': 'DejaVu Sans', 'weight': 'normal', 'size': 6}
plt.rc('font', **font)

os.makedirs(report_folder, exist_ok=True)

data = []
summary = dict()
for result in results:
  runs = {
      3: "Unrefined Dataset Improvement Run",
      4: "Manually Validated Dataset Improvement Run",
      6: "Refined Dataset Improvement Run"
  }

  method = result[0]
  elapsed = result[1]
  method_pretty = str(method).replace('_', ' ').title()
  title = f"{'Single Color' if no_multicolor else 'Multi-Color'} with {str(color_space).upper()} / {method_pretty}"

  y = result[2]
  yhat = result[3]

  report = classification_report(y, yhat, labels=range(len(classes)), target_names=classes, zero_division=0, output_dict=True)

  disp = ConfusionMatrixDisplay.from_predictions(y, yhat, labels=range(len(classes)), display_labels=classes, xticks_rotation='vertical', cmap=plt.cm.Blues, normalize=None)
  disp.figure_.set_dpi(150)
  disp.ax_.set_title(f"{method_pretty} [{elapsed:.2f} sec/Acc:{accuracy_score(y,yhat):.2f}]")
  disp.figure_.set_facecolor("white")
  disp.ax_.set_facecolor('white')
  disp.figure_.savefig(f"{report_folder}/{method}.png", dpi=130, bbox_inches='tight')

  data.append(dict(title=title, method=method, elapsed=elapsed, report=report))
  summary[method_pretty] = dict(duration=elapsed, accuracy=accuracy_score(y, yhat))

render = template.render(run=runs[dataset_id], data=data, summary=summary, color_space=color_space)


with open(f"{report_folder}/report.html", mode='w', encoding='utf-8') as file:
  file.write(render)

display(HTML(render))
